In [1]:
import pandas as pd
import numpy as np

from Models.grid_search_utils import GridSearch as Gs
from pandas import DataFrame

In [2]:
LGBMRanker = Gs.load_model(name="../Models/BlackBox/lmart/saved_models/LGBMRanker")

In [3]:
FIGSReg = Gs.load_model(name="../Models/GlassBox/figs/saved_models/FIGSReg")

In [4]:
EBMReg = Gs.load_model(name="../Models/GlassBox/ebm/saved_models/EBMReg")

In [5]:
def predict_rank(model, X:DataFrame, top_k:int=15):
    y_pred = model.predict(np.asarray(X.iloc[:,5:].values))
    y_pred = pd.DataFrame(y_pred, index=X.index, columns=["lambdas"])
    dt_final = pd.merge(X, y_pred, left_index=True, right_index=True)
    return dt_final.sort_values("lambdas",ascending=False)[["kId","lambdas","w_score","relevance"]].head(top_k)

In [6]:
test_set = pd.read_csv("../outputs/scores/0_dataset_ts.csv")

In [7]:
test_set[test_set["relevance"]==5]

,qId,kId,score,w_score,relevance,fitness_edu_basic,fitness_edu_bonus,fitness_city,fitness_age,fitness_exp_basic,fitness_exp_bonus,fitness_lang_basic,fitness_lang_bonus,fitness_comp_basic,fitness_comp_bonus,fitness_know_basic,fitness_knowl_bonus
72,34,2737,6.847489,0.702111,5,1.0,0.0,0.829537,1.0,1.0,0.0,0.50,0.3,1.000000,0.003667,0.714286,0.500000
78,33,2659,6.213733,0.667671,5,1.0,0.0,0.600943,1.0,1.0,0.0,1.00,0.0,0.592790,0.000000,0.520000,0.500000
101,88,6934,6.769328,0.681742,5,1.0,0.0,0.557738,1.0,1.0,0.0,1.00,0.0,1.000000,0.031685,1.000000,0.179905
109,15,1285,5.640950,0.673379,5,1.0,0.0,0.632364,1.0,0.0,0.0,1.00,0.0,1.000000,0.088936,0.515873,0.403776
131,5,470,6.652765,0.701582,5,1.0,0.0,0.519246,1.0,1.0,0.0,1.00,0.0,0.679322,0.263722,0.690476,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3053,51,4078,6.142267,0.674124,5,1.0,0.0,0.285153,1.0,1.0,0.0,0.85,0.0,1.000000,0.000000,1.000000,0.007114
3070,89,7052,7.388941,0.683669,5,1.0,0.0,0.999215,1.0,1.0,0.5,0.85,0.0,1.000000,0.022631,1.000000,0.017096
3083,5,432,7.206224,0.797179,5,1.0,0.0,0.886881,1.0,1.0,0.0,0.85,0.0,0.699839,0.269504,1.000000,0.500000
3114,28,2241,7.578540,0.713043,5,1.0,0.0,0.699921,1.0,1.0,0.5,1.00,0.0,1.007971,0.034572,1.000000,0.336075


In [8]:
job = test_set[test_set["qId"]==88]

# Perfect rank

In [9]:
job[["kId","w_score","relevance"]].sort_values("w_score",ascending=False).head(15)

,kId,w_score,relevance
548,6940,0.709451,5
371,6927,0.684909,5
1788,6956,0.684429,5
101,6934,0.681742,5
2322,6955,0.657881,4
2991,6962,0.637590,4
322,6967,0.636569,4
1642,6938,0.617930,4
1662,6946,0.599907,4
150,6943,0.591644,4


# LGBMRanker

In [10]:
predict_rank(LGBMRanker, job)

,kId,lambdas,w_score,relevance
548,6940,4.274854,0.709451,5
2322,6955,4.125146,0.657881,4
101,6934,3.922214,0.681742,5
2991,6962,3.692718,0.637590,4
1206,6960,3.506620,0.588671,4
371,6927,3.495735,0.684909,5
150,6943,3.424075,0.591644,4
1788,6956,2.483830,0.684429,5
2890,6933,2.148401,0.514181,3
316,6923,1.989241,0.580204,4


# EBM Regressor

In [11]:
predict_rank(EBMReg, job)

,kId,lambdas,w_score,relevance
548,6940,0.665539,0.709451,5
2322,6955,0.644997,0.657881,4
150,6943,0.640520,0.591644,4
101,6934,0.638325,0.681742,5
371,6927,0.629143,0.684909,5
2991,6962,0.625612,0.637590,4
329,6963,0.606195,0.551940,4
1206,6960,0.600545,0.588671,4
1662,6946,0.598408,0.599907,4
1788,6956,0.598177,0.684429,5


# FIGSRegressor

In [12]:
predict_rank(FIGSReg, job)

,kId,lambdas,w_score,relevance
548,6940,0.656423,0.709451,5
2322,6955,0.644402,0.657881,4
101,6934,0.641098,0.681742,5
371,6927,0.641098,0.684909,5
150,6943,0.641098,0.591644,4
1206,6960,0.612034,0.588671,4
2991,6962,0.612034,0.637590,4
1788,6956,0.608779,0.684429,5
1662,6946,0.602008,0.599907,4
329,6963,0.595405,0.551940,4
